In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
prep_config = {
    "typify": [
        {
            "name": "H",
            "charge": 0,
            "model_path": "/workspaces/salamander/units/peo_5mers.pdb",
            "model_top": "/workspaces/salamander/units/H.top",
        },
        {
            "name": "P",
            "charge": 0,
            "model_path": "/workspaces/salamander/units/peo_5mers.pdb",
            "model_top": "/workspaces/salamander/units/P.top",
        },
        {
            "name": "T",
            "charge": 0,
            "model_path": "/workspaces/salamander/units/peo_5mers.pdb",
            "model_top": "/workspaces/salamander/units/T.top",
        },
        {
            "name": "TFSI",
            "charge": -1,
            "model_path": "/workspaces/salamander/units/TFSI.pdb",
        },
    ],
    "polymerize": {
        "backend": "molpy",
        "structs": [
        {"name": "peo_25", "seq": ["H", *["P"] * 3, "T"]},
        {"name": "LiTFSI", "ions": ["TFSI", "LI"]},
    ],},
    "pack": {
        "structs": [
            {"name": "peo_25", "box": 200, "number": 100},
            {"name": "LiTFSI", "box": 200, "number": 20},
        ]
    },
}

md_eq_config = {
    "name": "md_eq",
    "stages": [
        {
            "type": "minimize",
        },
        {
            "type": "nvt",
            "Tstart": 400,
            "Tend": 400,
            "steps": 1000e3,
        },
        {
            "type": "npt",
            "Tstart": 400,
            "Tend": 400,
            "steps": 5000e3,
        },
        {
            "type": "npt",
            "Tstart": 400,
            "Tend": 1000,
            "steps": 2500e3,
        },
        {
            "type": "npt",
            "Tstart": 1000,
            "Tend": 550,
            "steps": 2500e3,
        },
        {"type": "npt", "Tstart": 550, "Tend": 50, "steps": 500 / 50 * 1e6},
    ]
}

process = [
    {"name": "prepare", "config": prep_config, "next": "md_eq"},
    {"name": "md_eq", "config": md_eq_config, "next": None},
]

In [16]:
from burr.core import action, State, ApplicationBuilder
from h_submitor import submit
from pathlib import Path

@action(reads=[], writes=['prep_dir'])
def prepare(state: State, prep_config:dict):
    import molexp as me
    import polyscan.app
    from hamilton.execution.executors import SynchronousLocalTaskExecutor

    dr = me.Task("peo_tg_test").with_modules(polyscan.app).enable_dynamic_execution(allow_experimental_mode=True).with_local_executor(SynchronousLocalTaskExecutor()).build()

    result = dr.execute(["pack"], inputs={"config": prep_config})
    return state.update(prep_dir=result['pack'])

@action(reads=['prep_dir'], writes=[])
@submit('slurm', 'slurm')
def md_eq(state: State, md_eq_config):
    import polyscan.eq_script
    prep_dir = state['prep_dir']
    md_eq_config['data_dir'] = prep_dir
    this_dir = Path('peo_tg_test') / prep_dir.stem
    this_dir.mkdir(parents=True, exist_ok=True)
    polyscan.eq_script.write_md_script(this_dir, md_eq_config)
    yield {
        'cmd': ['srun lmp -in eq.in'],
        "job_name": f'eq',
        "n_cores": 128,
        "run_time_max": '3-00:00:00',
        "account": 'project_465000925',
        "--partition": 'small',
        "cwd": this_dir,
    }
    # submit('slurm', 'slurm')(foo)()
    return state


app = ApplicationBuilder().with_actions(prepare, md_eq).with_transitions(["prepare", "md_eq"]).with_entrypoint("prepare").build()

app.run(halt_after=["md_eq"], inputs={"prep_config": prep_config, "md_eq_config": md_eq_config})


{'name': 'H', 'charge': 0, 'model_path': '/workspaces/salamander/units/peo_5mers.pdb', 'model_top': '/workspaces/salamander/units/H.top'}
{'name': 'P', 'charge': 0, 'model_path': '/workspaces/salamander/units/peo_5mers.pdb', 'model_top': '/workspaces/salamander/units/P.top'}
{'name': 'T', 'charge': 0, 'model_path': '/workspaces/salamander/units/peo_5mers.pdb', 'model_top': '/workspaces/salamander/units/T.top'}
{'name': 'TFSI', 'charge': -1, 'model_path': '/workspaces/salamander/units/TFSI.pdb'}
-I: Adding /opt/conda/dat/leap/prep to search path.
-I: Adding /opt/conda/dat/leap/lib to search path.
-I: Adding /opt/conda/dat/leap/parm to search path.
-I: Adding /opt/conda/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /opt/conda/dat/leap/cmd/leaprc.gaff
----- Source of /opt/conda/dat/leap/cmd/leaprc.gaff done
Log file: ./leap.log
Loading parameters: /opt/conda/dat/leap/parm/gaff.dat
Reading title:
AMBER Gene


********************************************************************************
-------------------------------------------------------------------
Oh no an error! Need help with Burr?
Join our discord and ask for help! https://discord.gg/4FxBMyzW5n
-------------------------------------------------------------------
> Action: `md_eq` encountered an error!<
> State (at time of action):
{'__PRIOR_STEP': 'prepare',
 '__SEQUENCE_ID': 1,
 'prep_dir': "PosixPath('peo_tg_test/.db/peo_25_n100_LiTFSI_n20'..."}
> Inputs (at time of action):
{'md_eq_config': "{'name': 'md_eq', 'stages': [{'type': 'minimize'},...",
 'prep_config': "{'typify': [{'name': 'H', 'charge': 0, 'model_path..."}
********************************************************************************
Traceback (most recent call last):
  File "/opt/conda/envs/work/lib/python3.10/site-packages/burr/core/application.py", line 885, in _step
    result, new_state = _run_single_step_action(
  File "/opt/conda/envs/work/lib/python3.10/s

FileNotFoundError: [Errno 2] No such file or directory: 'sbatch'